Neste exemplo, utilizamos o conjunto de dados CIFAR-10, que consiste em imagens de 10 classes diferentes. Primeiro, pré-processamos os dados normalizando-os para o intervalo [0, 1]. Em seguida, carregamos o modelo pré-treinado VGG16, que foi treinado na tarefa de classificação de imagens do ImageNet. Congelamos as camadas convolucionais do modelo pré-treinado para evitar que sejam treinadas novamente.

Em seguida, adicionamos camadas personalizadas no topo do modelo para adaptá-lo à tarefa de classificação de imagens do CIFAR-10. Compilamos o modelo com um otimizador Adam e a função de perda "sparse_categorical_crossentropy" adequada para problemas de classificação multiclasse.

Treinamos o modelo utilizando os dados do CIFAR-10, fazendo o fine-tuning apenas nas camadas adicionadas, mantendo as camadas convolucionais do VGG16 congeladas. Após o treinamento, avaliamos a acurácia do modelo utilizando os dados de teste.

O aprendizado por transferência permite aproveitar o conhecimento prévio aprendido por um modelo em uma tarefa relacionada e aplicá-lo a um novo problema, resultando em um treinamento mais rápido e melhor desempenho quando há uma quantidade limitada de dados disponíveis.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Carregar o conjunto de dados CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Pré-processamento dos dados
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Carregar o modelo pré-treinado VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Congelar as camadas convolucionais do modelo pré-treinado
for layer in base_model.layers:
    layer.trainable = False

# Adicionar camadas personalizadas no topo do modelo
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compilar o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo com aprendizado por transferência
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test))

# Avaliar a acurácia do modelo
_, accuracy = model.evaluate(X_test, y_test)
print("Acurácia:", accuracy)
